In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
import json
import glob2
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import regex as re
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch import optim
import os
import pickle
import random
!pip install transformers
from transformers import BertTokenizer   
from transformers import BertModel

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
     |████████████████████████████████| 675kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 16.6MB/s 
     |████████████████████████████████| 3.8MB 17.6MB/s 
     |████████████████████████████████| 890kB 38.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=48f905c24f8cfb11c0c941878b182852d3f6852725a5d8dca5d72b3fc032c2a2
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
class Node():
  def __init__(self,x=None,y=None,id=None,node1=None,node2=None):
    global gb
    if(id is None and (node1 is None or node2 is None)):
      raise Exception("To create non virtual nodes, id is required")
    if(id is not None):
      self.id=str(id)
      self.is_leaf_node=True
    else:
      gb.varint+=1
      self.id='vir_'+ str(gb.varint)
      self.node1=node1
      self.node2=node2  
      if('vir_' in node1.id and 'vir_' in node2.id):
        self.is_stance_node=False
      else:
        self.is_stance_node=True
      self.is_leaf_node=False  
    self.x=x
    self.y=y
    self.c=None
    self.h=None    

In [4]:
class Global:
    def __init__(self):
      self.varint=0
      self.g_parent=None
      self.g_nodes=[] 
    def reset(self):
      self.varint=0
      self.g_parent=None
      self.g_nodes=[]

def get_key_value(dicts,search_id,keys=None):
  res={}
  for d in dicts:
    if('id' in d.keys()):
      if(str(d['id'])==str(search_id)):
        for k in keys:
          res[k]=d[k]
        break
    else:
      if(search_id in d.keys()):  
        return d[search_id]
      else:
        return None        
  return list(res.values()) 

def uniform_embed(size):
  return np.random.uniform(low=-0.5, high=0.5, size=(size))

def create_tree(tree,tweets=None,stance_dict=None,parent=None,embed_size=2400):
  global gb
  children=[]
  for key,value in tree.items():
    if(type(value)==dict):      
      child=create_tree(value,tweets,stance_dict,parent=key,embed_size=embed_size)
    else:
      embed=get_key_value(dicts=tweets,search_id=key,keys=['text'])
      if(len(embed)>0 and len(embed[0])>0):
        embed=embed[0]
      else:
        embed=uniform_embed(embed_size)            
      child=Node(id=key,x=embed)
    children.append(child) 
  if(gb.g_parent is not None):
    arr=[gb.g_parent]
    arr.extend(children)
    children=arr 
  if(parent is not None):
    nodes=[]
    for child in children:
      if('vir_' not in str(parent) and 'vir_' in child.id):
        nodes.append(child)
        continue
      embed=get_key_value(dicts=tweets,search_id=parent,keys=['text']) 
      if(len(embed)>0 and len(embed[0])>0):
        embed=embed[0]  
      else:
        embed=uniform_embed(embed_size)      
      parent_node=Node(id=parent,x=embed) 
      stance=None
      if('var_' not in parent_node.id and 'var_' not in child.id):
        stance=get_key_value(dicts=[stance_dict],search_id=child.id) 
      node=Node(node1=parent_node,node2=child,y=stance)
      # print(parent,child.id,node.id)
      gb.g_nodes.extend([parent_node,child,node])
      nodes.append(node)
    node1=nodes[0] 
    for node_iter in range(1,len(nodes)):
      node2=nodes[node_iter]
      node=Node(node1=node1,node2=node2)
      # print(node1.id,node2.id,node.id)
      gb.g_nodes.extend([node])
      node1=node 
    embed=get_key_value(dicts=tweets,search_id=parent,keys=['text']) 
    if(len(embed)>0 and len(embed[0])>0):
      embed=embed[0]  
    else:
      embed=uniform_embed(embed_size)            
    gb.g_parent=Node(id=parent,x=embed)
    return node1
  return children     

# SKP Encoder python2

In [ ]:
# !git clone https://github.com/ryankiros/skip-thoughts.git
!wget http://www.cs.toronto.edu/~rkiros/models/dictionary.txt -P skip-thoughts
!wget http://www.cs.toronto.edu/~rkiros/models/utable.npy -P skip-thoughts
!wget http://www.cs.toronto.edu/~rkiros/models/btable.npy -P skip-thoughts
!wget http://www.cs.toronto.edu/~rkiros/models/uni_skip.npz -P skip-thoughts
!wget http://www.cs.toronto.edu/~rkiros/models/uni_skip.npz.pkl -P skip-thoughts
!wget http://www.cs.toronto.edu/~rkiros/models/bi_skip.npz -P skip-thoughts
!wget http://www.cs.toronto.edu/~rkiros/models/bi_skip.npz.pkl -P skip-thoughts

--2020-06-21 03:49:17--  http://www.cs.toronto.edu/~rkiros/models/dictionary.txt
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7996547 (7.6M) [text/plain]
Saving to: ‘skip-thoughts/dictionary.txt’

dictionary.txt      100%[===================>]   7.63M  11.0MB/s    in 0.7s    

2020-06-21 03:49:17 (11.0 MB/s) - ‘skip-thoughts/dictionary.txt’ saved [7996547/7996547]

--2020-06-21 03:49:19--  http://www.cs.toronto.edu/~rkiros/models/utable.npy
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2342138474 (2.2G)
Saving to: ‘skip-thoughts/utable.npy’

utable.npy          100%[===================>]   2.18G  84.4MB/s    in 28s     

2020-06-21 03:49:47 (79.0 MB/s) - ‘skip-thoughts/u

In [ ]:
# %cd skip-thoughts
# import importlib
# importlib.reload(skipthoughts)
import skipthoughts
model = skipthoughts.load_model()
encoder = skipthoughts.Encoder(model)

[Errno 2] No such file or directory: 'skip-thoughts'
/content/skip-thoughts


WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


# SKP encoder python3

In [ ]:
!wget "http://download.tensorflow.org/models/skip_thoughts_bi_2017_02_16.tar.gz"
!tar -xvf skip_thoughts_bi_2017_02_16.tar.gz
!rm skip_thoughts_bi_2017_02_16.tar.gz

--2020-06-21 06:02:01--  http://download.tensorflow.org/models/skip_thoughts_bi_2017_02_16.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 173.194.214.128, 2607:f8b0:400c:c0b::80
Connecting to download.tensorflow.org (download.tensorflow.org)|173.194.214.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5680270817 (5.3G) [application/x-gzip]
Saving to: ‘skip_thoughts_bi_2017_02_16.tar.gz’

skip_thoughts_bi_20 100%[===================>]   5.29G  58.6MB/s    in 74s     

2020-06-21 06:03:15 (72.9 MB/s) - ‘skip_thoughts_bi_2017_02_16.tar.gz’ saved [5680270817/5680270817]

skip_thoughts_bi_2017_02_16/
skip_thoughts_bi_2017_02_16/model.ckpt-500008.data-00000-of-00001
skip_thoughts_bi_2017_02_16/vocab.txt
skip_thoughts_bi_2017_02_16/model.ckpt-500008.index
skip_thoughts_bi_2017_02_16/embeddings.npy
skip_thoughts_bi_2017_02_16/model.ckpt-500008.meta


In [ ]:
!git clone https://github.com/elvisyjlin/skip-thoughts.git
%tensorflow_version 1.x
%cd skip-thoughts/

Cloning into 'skip-thoughts'...
remote: Enumerating objects: 35, done.
remote: Total 35 (delta 0), reused 0 (delta 0), pack-reused 35
Unpacking objects: 100% (35/35), done.
TensorFlow 1.x selected.
/content/skip-thoughts


In [ ]:
from skip_thoughts import configuration
from skip_thoughts import encoder_manager

VOCAB_FILE = "skip_thoughts_bi_2017_02_16/vocab.txt"
EMBEDDING_MATRIX_FILE = "skip_thoughts_bi_2017_02_16/embeddings.npy"
CHECKPOINT_PATH = "skip_thoughts_bi_2017_02_16/model.ckpt-500008"

encoder = encoder_manager.EncoderManager()
encoder.load_model(configuration.model_config(bidirectional_encoder=True),
                   vocabulary_file=VOCAB_FILE,
                   embedding_matrix_file=EMBEDDING_MATRIX_FILE,
                   checkpoint_path=CHECKPOINT_PATH)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.


INFO:tensorflow:Reading vocabulary from skip_thoughts_bi_2017_02_16/vocab.txt

INFO:tensorflow:Loaded vocabulary with 930914 words.
INFO:tensorflow:Loading embedding matrix from skip_thoughts_bi_2017_02_16/embeddings.npy
INFO:tensorflow:Loaded embedding matrix with shape (930914, 620)
INFO:tensorflow:Building model.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TFRecordDataset`.


Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updat

# DeepMoji Encoder

In [ ]:
# for deepMoji
!git clone https://github.com/zzsza/DeepMoji-Python3.git
%cd DeepMoji-Python3/DeepMoji-master
!python scripts/download_weights.py
!pip install emoji
# replace downloaded encode_texts.py file with modified file

Cloning into 'DeepMoji-Python3'...
remote: Enumerating objects: 85, done.
remote: Total 85 (delta 0), reused 0 (delta 0), pack-reused 85
Unpacking objects: 100% (85/85), done.
/content/DeepMoji-Python3/DeepMoji-master
About to download the pretrained weights file from https://www.dropbox.com/s/xqarafsl6a8f9ny/deepmoji_weights.hdf5?dl=0#
The size of the file is roughly 85MB. Continue? [y/n]
y
Downloading...
Running system call: wget https://www.dropbox.com/s/xqarafsl6a8f9ny/deepmoji_weights.hdf5?dl=0# -O /content/DeepMoji-Python3/DeepMoji-master/model/deepmoji_weights.hdf5
--2020-06-22 07:25:53--  https://www.dropbox.com/s/xqarafsl6a8f9ny/deepmoji_weights.hdf5?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.67.1, 2620:100:6023:1::a27d:4301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.67.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/xqarafsl6a8f9ny/deepmoji_weights.hdf5 [following]
--2020-06-22 07:25:53--  https

# BERT Encoder

In [5]:
class SentenceClassifier(nn.Module):
  def __init__(self):
      super(SentenceClassifier, self).__init__()
      self.bert_layer = BertModel.from_pretrained('/content/drive/My Drive/BERT_CONFIG')
      self.linear = nn.Linear(768*2, 3)
      self.softmax = nn.Softmax()

  def forward(self,seq,mask):
      _,cls = self.bert_layer(seq, attention_mask = mask)
      return cls.tolist()

# Encoders

In [6]:
# import examples.encode_texts

class GLOVE:
  def __init__(self,embedding_file):
    self.vocab={}
    self.unk=[0 for i in range(300)]
    with open(embedding_file, 'rt', encoding='utf-8') as f:
      for line in f:
        splitline=line.split()
        self.vocab[splitline[0]]=[float(value) for value in splitline[1:]]
    self.size=300    
  def embed(self,tokens):
    embed = [self.vocab.get(token,self.unk) for token in tokens]
    embed = [sum(x)/len(x) for x in zip(*embed)]
    return embed

class DeepMoji:
  def __init__(self):
    self.size=2304
  def embed(self,tokens):
    embed=examples.encode_texts.encode(np.array([''.join(tokens)]))
    return embed.reshape(2304)

class SKP:
  def __init__(self,encoder):
    self.encoder=encoder
    self.size=2400
  def embed(self,tokens):
    embed=self.encoder.encode(tokens) 
    embed=[sum(x)/len(x) for x in zip(*embed)]
    return embed

class skpemt:
  def __init__(self,encoder):
    self.encoder=encoder
    self.size=2400+2304
  def embed(self,tokens):
    encoding=self.encoder.encode(tokens) 
    encoding=[sum(x)/len(x) for x in zip(*encoding)]
    deepmoji_encoding=examples.encode_texts.encode(np.array([''.join(tokens)]))
    deepmoji_encoding= list(deepmoji_encoding.reshape(2304))
    encoding.extend(deepmoji_encoding)
    return encoding 

class BERT:
  def __init__(self):
    self.model=SentenceClassifier()
    self.model.load_state_dict(torch.load('/content/drive/My Drive/BERT_CONFIG/multinli_model.pt'))
    self.size=768
    self.tokenizer = BertTokenizer.from_pretrained('/content/drive/My Drive/BERT_CONFIG')    
  def tokenize(self,tokens):
    return [self.tokenizer.convert_tokens_to_ids('[CLS]')]\
      +[self.tokenizer.convert_tokens_to_ids(tok) for tok in tokens]
  def embed(self,tokens):
    tokenized=torch.tensor(self.tokenize(tokens)).view(1,-1)
    mask=torch.tensor([1 for i in range(tokenized.size(-1))]).view(1,-1)
    return self.model(tokenized,mask)            

# Preprocessing Input

In [8]:
def create_datapoint(source,reply_list,struct,stance_dict,embedding):
    tweets=reply_list+[source]
    create_tree(struct,tweets,stance_dict,embed_size=embedding.size)

def preprocess(tweet):
    # remove @mentions, RT,MT,DM,PRT,HT,CC, URLs
    contractions = { 
      "n't": "not",
      "'ve": "have",
      "'d": "would",
      "'ll": "will",
      "'m": "am",
      "ma'am": "madam",
      "'re": "they are"
      }
    text=tweet.lower()  
    tokens=text.split()
    # print(tokens)
    for indx,token in enumerate(tokens):
        for contra in contractions.keys():
            if(contra in token):
                tokens[indx]=' '.join([token.replace(contra,''),contractions[contra]])
    text=' '.join(tokens)
    # Format words and remove unwanted characters    
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'([@?])(\w+)\b', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text=text.replace('mt ',' ').replace('rt ',' ').replace('dm ',' ').replace('prt ',' ').replace('ht ',' ').replace('cc ',' ')
    tokens = text.split()
    stops = set(stopwords.words("english"))
    tokens = [w for w in tokens if not w in stops]
    return tokens

In [13]:
text='MT @euronews France: 10 dead after shooting at HQ of satirical weekly #CharlieHebdo. If Zionists/Jews did this they\'d be nuking Israel'
preprocess(text)

['france',
 '10',
 'dead',
 'shooting',
 'hq',
 'satirical',
 'weekly',
 'charliehebdo',
 'zionists',
 'jews',
 'would',
 'nuking',
 'israel']

# Model

In [7]:
class TreeLSTMCell(nn.Module):
  def __init__(self,x_size,h_size):
    super(TreeLSTMCell, self).__init__()
    self.W_iou = nn.Linear(x_size, 3 * h_size, bias=False)
    self.U_iou = nn.Linear(h_size, 3 * h_size, bias=False)
    self.b_iou = nn.Parameter(torch.zeros(1, 3 * h_size))
    self.U_f = nn.Linear(2 * h_size, 2 * h_size)
    self.b_f = nn.Parameter(torch.zeros(1, 2 * h_size))      
    self.h=torch.zeros(1,h_size)
    self.c=torch.zeros(1,h_size)
    self.sigmoid=nn.Sigmoid()
    self.tanh=nn.Tanh()
    self.conv=nn.Conv2d(in_channels=1,out_channels=1, kernel_size=2, stride=1,padding=(0,1))
    self.maxpool=nn.MaxPool2d(kernel_size=(1,2),stride=1)
    self.stance_linear=nn.Linear(h_size,4)
    self.veracity_linear=nn.Linear(h_size,3)
    # self.softmax=nn.LogSoftmax(dim=-1)
    self.h_size=h_size
  def reset_h_c(self):
    self.h=torch.zeros(1,self.h_size)
    self.c=torch.zeros(1,self.h_size)    
  def forward(self,node,is_last=False):      
    if(node.is_leaf_node):
      x=torch.tensor(node.x,dtype=torch.float)
      x_iou=self.W_iou(x.view(1,-1)) 
      u_iou=self.U_iou(self.h)
      b_iou=self.b_iou      
      xs=torch.chunk(x_iou,3,-1)
      us=torch.chunk(u_iou,3,-1)
      bs=torch.chunk(b_iou,3,-1)       
      i=self.sigmoid(xs[0]+us[0]+bs[0])
      o=self.sigmoid(xs[1]+us[1]+bs[1])
      u=self.sigmoid(xs[2]+us[2]+bs[2])
      self.c=i*u
      self.h=o*self.tanh(self.c)
      node.h=self.h
      node.c=self.c
    else:
      h_stack=torch.stack([node.node1.h,node.node2.h])
      u_f=self.U_f(h_stack.view(1,-1))
      f=self.sigmoid(u_f+self.b_f)
      h_hat=self.maxpool(self.conv(h_stack.permute(1,0,2).unsqueeze(0))).view(1,-1)
      u_iou=self.U_iou(h_hat)
      us=torch.chunk(u_iou,3,-1)
      bs=torch.chunk(self.b_iou,3,-1)            
      i=self.sigmoid(us[0]+bs[0])
      o=self.sigmoid(us[1]+bs[1])
      u=self.sigmoid(us[2]+bs[2])
      self.c=i*u + torch.sum(f.view(-1,self.h_size)*torch.stack([node.node1.c,node.node2.c]).view(-1,self.h_size),0)
      self.h=o*self.tanh(self.c)
      node.h=self.h
      node.c=self.c
      if(node.is_stance_node):
        stance=self.stance_linear(self.h)
        return stance        
      if(is_last):
        veracity=self.veracity_linear(self.h)
        return veracity  

# Loading Data

In [8]:
veracity_enum = { 
  'true':0,
  'false':1,
  'unverified':2
}

stance_enum={
    'agreed':0,
    'comment':1,
    'appeal-for-more-information':2,
    'disagreed':3
}

In [9]:
class Event:
  def __init__(self,path):
    self.path=path
    self.treelist=[]
  def create_tree(self,gb):
    dirs=os.listdir(self.path)
    for d in dirs:
      if(d=='.DS_Store'):
        continue
      source=glob2.glob(self.path+'/'+ d + '/source-tweets/*.json')[0]
      replies=glob2.glob(self.path+'/'+ d + '/reactions/*.json')
      structure=self.path+'/'+ d + '/structure.json'
      gb.reset()
      reply_list=[]
      with open(source,'r') as f:
        source=json.load(f,parse_int=None)
      processed_text= preprocess(source['text'])  
      if(len(processed_text)>0):  
        source={'id':source['id'],'text':embedding.embed(processed_text)}  
      else:
        source={'id':source['id'],'text':processed_text}        
      for reply in replies:
        with open(reply,'r') as f:
          reply=json.load(f,parse_int=None)
        processed_text= preprocess(reply['text'])
        if(len(processed_text)>0):
          reply={'id':reply['id'],'text':embedding.embed(processed_text)}
        else:
          reply={'id':reply['id'],'text':processed_text} 
        reply_list.append(reply)
        # print(reply_list)      
      with open(structure,'r') as f:
        struct=json.load(f,parse_int=None)
      create_datapoint(source,reply_list,struct,stance_dict,embedding=embedding)
      self.treelist.append((source['id'],gb.g_nodes))      

In [10]:
import ast
event_labels=['charliehebdo','sydneysiege','germanwings-crash','ferguson','ottawashooting']
# load stance labels
# rumour_files=glob2.glob('/content/drive/My Drive/rumoureval-data/traindev/*subtaskA*.json')
# stance_dict={}
# for file in rumour_files:
#   with open(file,'r') as f:
#     stance_dict.update(json.load(f,parse_int=None))
# for k,v in stance_dict.items():
#   stance_dict[k]=stance_enum[v]

def convert_annotations(annotation, string = True):
    if 'misinformation' in annotation.keys() and 'true'in annotation.keys():
        if int(annotation['misinformation'])==0 and int(annotation['true'])==0:
            if string:
                label = "unverified"
            else:
                label = 2
        elif int(annotation['misinformation'])==0 and int(annotation['true'])==1 :
            if string:
                label = "true"
            else:
                label = 1
        elif int(annotation['misinformation'])==1 and int(annotation['true'])==0 :
            if string:
                label = "false"
            else:
                label = 0
        elif int(annotation['misinformation'])==1 and int(annotation['true'])==1:
            print ("OMG! They both are 1!")
            print(annotation['misinformation'])
            print(annotation['true'])
            label = None
            
    elif 'misinformation' in annotation.keys() and 'true' not in annotation.keys():
        # all instances have misinfo label but don't have true label
        if int(annotation['misinformation'])==0:
            if string:
                label = "unverified"
            else:
                label = 2
        elif int(annotation['misinformation'])==1:
            if string:
                label = "false"
            else:
                label = 0
                
    elif 'true' in annotation.keys() and 'misinformation' not in annotation.keys():
        print ('Has true not misinformation')
        label = None
    else:
        print('No annotations')
        label = None
           
    return label
#load veracity labels
veracity_dict={}
root_dir='/content/drive/My Drive/PHEME'
for e in event_labels:
  dirs=os.listdir(root_dir + '/{}'.format(e))
  for d in dirs:
    if(d=='.DS_Store'):
      continue
    with open(root_dir+'/'+e+'/'+d+ '/annotation.json','r') as f:
      veracity_dict.update({d:veracity_enum[convert_annotations(json.load(f))]})

# load stance labels
stance_dict={}
stance_annotation_file=root_dir+'/annotations/en-scheme-annotations.json'
with open(stance_annotation_file,'r') as f:
  lines=f.readlines()
for line in lines:
  if('#' in line):
    continue
  annot=ast.literal_eval(line)
  if('responsetype-vs-source' in annot.keys()):
    stance_dict[annot['tweetid']]=stance_enum[annot['responsetype-vs-source']]
  elif('responsetype-vs-previous' in annot.keys()):
    stance_dict[annot['tweetid']]=stance_enum[annot['responsetype-vs-previous']]

# veracity_dict={}
# for file in rumour_files:
#   with open(file,'r') as f:
#     veracity_dict.update(json.load(f,parse_int=None))
# for k,v in veracity_dict.items():
#   veracity_dict[k]=veracity_enum[v]

In [ ]:
embedding=BERT()
with open('/content/drive/My Drive/rumoureval-data/charliehebdo/553184482241814530/replies/553184702530453505.json','r') as f:
  source=json.load(f,parse_int=None)
source={'id':source['id'],'text':embedding.embed(preprocess(source['text']))} 
source
# type(encoder.encode(['maybe', 'they', 'want', 'to', 'get' ,'caught']))

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,


{'id': 553184702530453505,
 'text': tensor([[-0.3701, -0.2217,  0.9532,  0.5189,  0.8301,  0.2629,  0.8415,  0.6777,
           0.9937, -0.9796,  0.9742, -0.8071,  0.9026, -0.2117,  0.9733, -0.5808,
           0.0143, -0.3522,  0.3882, -0.8300,  0.7825,  0.0173,  0.8494,  0.6325,
           0.3077, -0.8371, -0.4423,  0.9895,  0.9898,  0.4993, -0.1705, -0.5002,
          -0.8806,  0.4054,  0.8998, -0.1324, -0.6142, -0.3484, -0.4314,  0.5374,
          -0.9941,  0.6816,  0.7429, -0.9557,  0.6775,  0.0691, -0.0355, -0.3049,
          -0.6994, -0.9876, -0.4933, -0.8455, -0.5568,  0.0603, -0.3631, -0.1101,
          -0.6004, -0.7368, -0.5735,  0.1829,  0.0956, -0.5329,  0.7888, -0.8245,
          -0.9179, -0.9017, -0.1838, -0.5882, -0.3484, -0.6698,  0.4890, -0.0192,
           0.2107, -0.9385, -0.8362, -0.6202, -0.2919,  0.2919,  0.1369, -0.9912,
          -0.9393, -0.8425, -0.2211,  0.8951, -0.6955,  0.0378, -0.8312,  0.2405,
          -0.9967,  0.0448,  0.6354,  0.3575, -0.5427,  0.5747,

In [23]:
import pickle
# import importlib
# importlib.reload(examples.encode_texts)

def dump_embedding(typ):
  root_path='/content/drive/My Drive/PHEME/'
  for e in ['charliehebdo','sydneysiege','germanwings-crash','ferguson','ottawashooting']:
  # for e in ['putinmissing','ebola-essien','prince-toronto']:
    # event_list=[]
    ev=Event(root_path+e)
    ev.create_tree(gb)
    # event_list.append(ev)
    with open('/content/drive/My Drive/PHEME/Encoding/{}/data_{}.pkl'.format(typ,e),'wb') as f:
      pickle.dump(ev,f)

# embedding=SKP(encoder)
# embedding=DeepMoji()
embedding=BERT()
gb=Global()
# dump_embedding('skp')
dump_embedding('BERT')      

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,


In [ ]:
[(node.id,node.x) for node in gb.g_nodes]
# with open('/content/drive/My Drive/data.pkl','rb') as f:
#   event_list=pickle.load(f)

# Veracity Tree Balancing

In [38]:
# Balancing each class for rumour labels
def balance_embedding(typ):
  ev_label_count=[]
  for e in ['charliehebdo','sydneysiege','germanwings-crash','ferguson','ottawashooting']:
    with open('/content/drive/My Drive/PHEME/Encoding/{}/data_{}.pkl'.format(typ,e),'rb') as f:
      ev=pickle.load(f)
    label={0:0,1:0,2:0}   
    # for event in ev:
    for tree in ev.treelist:
      label[veracity_dict[str(tree[0])]]+=1
    ev_label_count.append((e,label))    
  print(ev_label_count)
  for e in ['charliehebdo','sydneysiege','germanwings-crash','ferguson','ottawashooting']:
    with open('/content/drive/My Drive/PHEME/Encoding/{}/data_{}.pkl'.format(typ,e),'rb') as f:
      ev=pickle.load(f) 
    # for event in ev:
    lbl_0_list=[]
    lbl_1_list=[]
    lbl_2_list=[]
    for tree in ev.treelist: 
      if(veracity_dict[str(tree[0])]==0):
        lbl_0_list.append(tree)      
      elif(veracity_dict[str(tree[0])]==1):
        lbl_1_list.append(tree)
      elif(veracity_dict[str(tree[0])]==2):
        lbl_2_list.append(tree)
    if(e=='charliehebdo'):
      for i in range(27):
        ev.treelist.append(random.sample(lbl_1_list,1)[0])
      for i in range(16):
        ev.treelist.append(random.sample(lbl_2_list,1)[0])
    elif(e=='sydneysiege'):
      for i in range(38):
        ev.treelist.append(random.sample(lbl_1_list,1)[0])
      for i in range(44):
        ev.treelist.append(random.sample(lbl_2_list,1)[0])   
    elif(e=='germanwings-crash'):
      for i in range(9):
        ev.treelist.append(random.sample(lbl_2_list,1)[0]) 
    elif(e=='ferguson'):
      for i in range(42):
        ev.treelist.append(random.sample(lbl_0_list,1)[0])
    elif(e=='ottawashooting'):
      for i in range(40):
        ev.treelist.append(random.sample(lbl_1_list,1)[0])    
      for i in range(23):
        ev.treelist.append(random.sample(lbl_2_list,1)[0])    
    with open('/content/drive/My Drive/PHEME/Encoding/{}/data_{}_balanced.pkl'.format(typ,e),'wb') as f:
      pickle.dump(ev,f) 

  ev_label_count=[]
  for e in ['charliehebdo','sydneysiege','germanwings-crash','ferguson','ottawashooting']:
    with open('/content/drive/My Drive/PHEME/Encoding/{}/data_{}_balanced.pkl'.format(typ,e),'rb') as f:
      ev=pickle.load(f)
    label={0:0,1:0,2:0}   
    # for event in ev:
    for tree in ev.treelist:
      label[veracity_dict[str(tree[0])]]+=1
    ev_label_count.append((e,label))    
  print(ev_label_count) 

# balance_embedding('skp')                                                                   
# balance_embedding('deepmoji')
balance_embedding('BERT')                                                                   

[('charliehebdo', {0: 39, 1: 12, 2: 23}), ('sydneysiege', {0: 51, 1: 13, 2: 7}), ('germanwings-crash', {0: 10, 1: 12, 2: 3}), ('ferguson', {0: 2, 1: 0, 2: 44}), ('ottawashooting', {0: 35, 1: 10, 2: 13})]
[('charliehebdo', {0: 39, 1: 39, 2: 39}), ('sydneysiege', {0: 51, 1: 51, 2: 51}), ('germanwings-crash', {0: 10, 1: 12, 2: 12}), ('ferguson', {0: 44, 1: 0, 2: 44}), ('ottawashooting', {0: 35, 1: 50, 2: 36})]


# Data Loader

In [13]:
event_labels=['charliehebdo','sydneysiege','germanwings-crash','ferguson','ottawashooting']
# change type for embedding skp,deepmoji,bert,mix(deepmoji+skp)
def train_loader(val_index):
  for e in [l for i,l in enumerate(event_labels) if i!=val_index]:
    with open('/content/drive/My Drive/PHEME/Encoding/BERT/data_{}_balanced.pkl'.format(e),'rb') as f:
      ev=pickle.load(f)
    yield ev,e

def val_loader(val_index):
  for e in [l for i,l in enumerate(event_labels) if i==val_index]:
    with open('/content/drive/My Drive/PHEME/Encoding/BERT/data_{}.pkl'.format(e),'rb') as f:
      ev=pickle.load(f)
    yield ev,e

# Training without stance loss averaging

In [15]:
# Training on unbalanced data without loss averaging for stance
model=TreeLSTMCell(768,768)

optimizer = optim.Adam(model.parameters(), lr=.008)
criterion = nn.CrossEntropyLoss()

total_loss=[]
for e in range(5):  
  print('==================== Iteration {} =========='.format(e))
  val_index=e%5
  train_loss=0
  for event,lbl in train_loader(val_index):
    print('training on {}...'.format(lbl))    
    for tree in event.treelist:
      loss=0
      stance_loss=0
      optimizer.zero_grad()  
      nodes=tree[1]
      for i in range(len(nodes)):
        nodes[i].h=None
        nodes[i].c=None
      model.reset_h_c()  
      for i in range(len(nodes)):
        if(i!=(len(nodes)-1)):
          result=model(nodes[i])
          if(result is not None and nodes[i].y is not None):
            stance_loss+=criterion(result,torch.tensor(nodes[i].y).view(-1))            
        else:
          result=model(nodes[i],is_last=True)       
          loss+=criterion(result,torch.tensor(veracity_dict[str(tree[0])],dtype=torch.long).view(-1))   
      loss+=stance_loss            
      train_loss+=loss.data.item()        
      loss.backward()
      optimizer.step()    
    print('train loss : {}'.format(train_loss))     
  with torch.no_grad():
    validation_loss=0
    for event,lbl in val_loader(val_index):
      print('testing on {}...'.format(lbl))    
      for tree in event.treelist:
        loss=0
        nodes=tree[1]
        for i in range(len(nodes)):
          nodes[i].h=None
          nodes[i].c=None
        model.reset_h_c()  
        for i in range(len(nodes)):
          if(i!=(len(nodes)-1)):
            result=model(nodes[i])
            if(result is not None and nodes[i].y is not None):
              stance_loss+=criterion(result,torch.tensor(nodes[i].y).view(-1))
          else:
            result=model(nodes[i],is_last=True)      
            loss+=criterion(result,torch.tensor(veracity_dict[str(tree[0])],dtype=torch.long).view(-1))  
        loss+=stance_loss            
        validation_loss+=loss.data.item()   
      print('validation loss : {}'.format(validation_loss))
      torch.save(model.state_dict(), '/content/drive/My Drive/model_bert_balanced_30.pt')    

==================== Iteration 0 ==========
training on sydneysiege...
train loss : 2890.2772336006165
training on germanwings-crash...
train loss : 3314.25519824028
training on ferguson...
train loss : 5115.741343259811
training on ottawashooting...
train loss : 6866.604239463806
testing on charliehebdo...
validation loss : 38604.97555541992
==================== Iteration 1 ==========
training on charliehebdo...
train loss : 1539.7985394001007
training on germanwings-crash...
train loss : 1934.5494248867035
training on ferguson...
train loss : 3711.055954694748
training on ottawashooting...
train loss : 5434.356980085373
testing on sydneysiege...
validation loss : 43696.31030654907
==================== Iteration 2 ==========
training on charliehebdo...
train loss : 1532.8338586091995
training on sydneysiege...
train loss : 4093.521251797676
training on ferguson...
train loss : 5890.21211540699
training on ottawashooting...
train loss : 7619.407017469406
testing on germanwings-crash...

# Training with stance loss averaging

In [44]:
# skp:
# model=TreeLSTMCell(2400,2400)
# deepmoji
model=TreeLSTMCell(768,768)

optimizer = optim.Adam(model.parameters(), lr=.001)
criterion = nn.CrossEntropyLoss()

# validation_losses={'charliehebdo':0,'sydneysiege':0,'germanwings-crash':0,'ferguson':0,'ottawashooting':0}
total_loss=[]
for e in range(5):  
  print('==================== Iteration {} =========='.format(e))
  val_index=e%5
  train_loss=0
  for event,lbl in train_loader(val_index):
    print('training on {}...'.format(lbl))    
    for tree in event.treelist:
      loss=0
      stance_loss=0
      optimizer.zero_grad()      
      stance_loss_0=[]
      stance_loss_1=[]
      stance_loss_2=[]
      stance_loss_3=[]
      nodes=tree[1]
      for i in range(len(nodes)):
        nodes[i].h=None
        nodes[i].c=None
      model.reset_h_c()  
      for i in range(len(nodes)):
        if(i!=(len(nodes)-1)):
          result=model(nodes[i])
          if(result is not None and nodes[i].y is not None):
            if(e>28):
              print('stance : {}'.format(result),nodes[i].y)
            if(nodes[i].y==0):
              stance_loss_0.append(criterion(result,torch.tensor(nodes[i].y).view(-1)))
            if(nodes[i].y==1):
              stance_loss_1.append(criterion(result,torch.tensor(nodes[i].y).view(-1)))
            if(nodes[i].y==2):
              stance_loss_2.append(criterion(result,torch.tensor(nodes[i].y).view(-1)))
            if(nodes[i].y==3):
              stance_loss_3.append(criterion(result,torch.tensor(nodes[i].y).view(-1)))
        else:
          result=model(nodes[i],is_last=True)  
          # print(result,veracity_dict[str(tree[0])],nodes[i].id)        
          loss+=criterion(result,torch.tensor(veracity_dict[str(tree[0])],dtype=torch.long).view(-1))  
      if(len(stance_loss_0)>0):
        stance_loss_0=sum(stance_loss_0)/len(stance_loss_0)
      else:
        stance_loss_0=0   
      if(len(stance_loss_1)>0):
        stance_loss_1=sum(stance_loss_1)/len(stance_loss_1)
      else:
        stance_loss_1=0        
      if(len(stance_loss_2)>0):
        stance_loss_2=sum(stance_loss_2)/len(stance_loss_2)
      else:
        stance_loss_2=0        
      if(len(stance_loss_3)>0):
        stance_loss_3=sum(stance_loss_3)/len(stance_loss_3)
      else:
        stance_loss_3=0   
      # if(e>28):      
      #   print('stance0 loss: {}'.format(stance_loss_0),'stance1 loss: {}'.format(stance_loss_1),\
      #         'stance2 loss: {}'.format(stance_loss_2),'stance3 loss: {}'.format(stance_loss_3))
      stance_loss+=stance_loss_0+stance_loss_1+stance_loss_2+stance_loss_3
    # if(e>28):
    #   print('stance_loss:{}'.format(stance_loss))
      loss+=stance_loss            
      train_loss+=loss.data.item()        
      loss.backward()
      optimizer.step()    
    print('train loss : {}'.format(train_loss))     
  with torch.no_grad():
    validation_loss=0
    for event,lbl in val_loader(val_index):
      print('testing on {}...'.format(lbl))    
      for tree in event.treelist:
        loss=0
        stance_loss=0        
        stance_loss_0=[]
        stance_loss_1=[]
        stance_loss_2=[]
        stance_loss_3=[]
        nodes=tree[1]
        for i in range(len(nodes)):
          nodes[i].h=None
          nodes[i].c=None
        model.reset_h_c()  
        for i in range(len(nodes)):
          if(i!=(len(nodes)-1)):
            result=model(nodes[i])
            if(result is not None and nodes[i].y is not None):
              # if(e>28):
              #   print('stance : {}'.format(result),nodes[i].y)
              if(nodes[i].y==0):
                stance_loss_0.append(criterion(result,torch.tensor(nodes[i].y).view(-1)))
              if(nodes[i].y==1):
                stance_loss_1.append(criterion(result,torch.tensor(nodes[i].y).view(-1)))
              if(nodes[i].y==2):
                stance_loss_2.append(criterion(result,torch.tensor(nodes[i].y).view(-1)))
              if(nodes[i].y==3):
                stance_loss_3.append(criterion(result,torch.tensor(nodes[i].y).view(-1)))
          else:
            result=model(nodes[i],is_last=True)  
            # print(result,veracity_dict[str(tree[0])],nodes[i].id)        
            loss+=criterion(result,torch.tensor(veracity_dict[str(tree[0])],dtype=torch.long).view(-1))  
        if(len(stance_loss_0)>0):
          stance_loss_0=sum(stance_loss_0)/len(stance_loss_0)
        else:
          stance_loss_0=0   
        if(len(stance_loss_1)>0):
          stance_loss_1=sum(stance_loss_1)/len(stance_loss_1)
        else:
          stance_loss_1=0        
        if(len(stance_loss_2)>0):
          stance_loss_2=sum(stance_loss_2)/len(stance_loss_2)
        else:
          stance_loss_2=0        
        if(len(stance_loss_3)>0):
          stance_loss_3=sum(stance_loss_3)/len(stance_loss_3)
        else:
          stance_loss_3=0   
        # if(e>28):      
        #   print('stance0 loss: {}'.format(stance_loss_0),'stance1 loss: {}'.format(stance_loss_1),\
        #         'stance2 loss: {}'.format(stance_loss_2),'stance3 loss: {}'.format(stance_loss_3))
        stance_loss+=stance_loss_0+stance_loss_1+stance_loss_2+stance_loss_3
      # if(e>28):
      #   print('stance_loss:{}'.format(stance_loss))
        loss+=stance_loss            
        validation_loss+=loss.data.item()   
      print('validation loss : {}'.format(validation_loss))
      
    # if(e>0 and e%5==0): 
    #   sum_loss=sum([v for k,v in validation_loss])
      # if(e/5>0 and validation_loss<validation_losses[lbl]):
      torch.save(model.state_dict(), '/content/drive/My Drive/model_bert_balanced_stance_averaged.pt')
      # else:
        # exit()        
      # validation_losses[lbl]=validation_loss         
      # total_loss.append(sum_loss)      

==================== Iteration 0 ==========
training on sydneysiege...
train loss : 754.097038269043
training on germanwings-crash...
train loss : 955.8442323207855
training on ferguson...
train loss : 1415.5786092281342
training on ottawashooting...
train loss : 1994.668744623661
testing on charliehebdo...
validation loss : 591.5271751880646
==================== Iteration 1 ==========
training on charliehebdo...
train loss : 538.2599716186523
training on germanwings-crash...
train loss : 688.7457664012909
training on ferguson...
train loss : 1161.300347685814
training on ottawashooting...
train loss : 1716.704894900322
testing on sydneysiege...
validation loss : 895.4520432949066
==================== Iteration 2 ==========
training on charliehebdo...
train loss : 526.8075971603394
training on sydneysiege...
train loss : 1295.2107627391815
training on ferguson...
train loss : 1729.2560676336288
training on ottawashooting...
train loss : 2275.6516404747963
testing on germanwings-crash..

# Metrics

In [16]:
def val_loader(val_index):
  for e in [l for i,l in enumerate(event_labels) if i==val_index]:
    with open('/content/drive/My Drive/PHEME/Encoding/BERT/data_{}.pkl'.format(e),'rb') as f:
      ev=pickle.load(f)
    yield ev,e

model=TreeLSTMCell(768,768)
model.load_state_dict(torch.load('/content/drive/My Drive/model_bert_balanced_30.pt'))
results={'stance':{'charliehebdo':[],'sydneysiege':[],\
                    'germanwings-crash':[],'ferguson':[],'ottawashooting':[]}, \
                    'veracity':{'charliehebdo':[],'sydneysiege':[],\
                    'germanwings-crash':[],'ferguson':[],'ottawashooting':[]}}
with torch.no_grad():
  for val_index in range(5):
    for event,lbl in val_loader(val_index):
      print('testing on {}...'.format(lbl))    
      for tree in event.treelist:
        nodes=tree[1]
        for i in range(len(nodes)):
          nodes[i].h=None
          nodes[i].c=None
        model.reset_h_c()  
        for i in range(len(nodes)):
          if(i!=(len(nodes)-1)):
            result=model(nodes[i])
            if(result is not None and nodes[i].y is not None):
              results['stance'][lbl].append((nodes[i].y,torch.argmax(result,dim=-1).item()))
          else:
            result=model(nodes[i],is_last=True) 
            results['veracity'][lbl].append((veracity_dict[str(tree[0])],torch.argmax(result,dim=-1).item()))              

testing on charliehebdo...
testing on sydneysiege...
testing on germanwings-crash...
testing on ferguson...
testing on ottawashooting...


In [17]:
from sklearn.metrics import classification_report
for lbl in ['charliehebdo','sydneysiege','germanwings-crash','ferguson','ottawashooting']:
  y_true=[y[0] for y in results['veracity'][lbl]]
  y_pred=[y[1] for y in results['veracity'][lbl]]
  print('=============={}==Veracity Report=============='.format(lbl))
  print(classification_report(y_true,y_pred))

  y_true=[y[0] for y in results['stance'][lbl]]
  y_pred=[y[1] for y in results['stance'][lbl]]
  print('=============={}==Stance Report=============='.format(lbl))
  print(classification_report(y_true,y_pred)) 

==============charliehebdo==Veracity Report==============
              precision    recall  f1-score   support

           0       0.53      1.00      0.69        39
           1       0.00      0.00      0.00        12
           2       0.00      0.00      0.00        23

    accuracy                           0.53        74
   macro avg       0.18      0.33      0.23        74
weighted avg       0.28      0.53      0.36        74

==============charliehebdo==Stance Report==============
              precision    recall  f1-score   support

           0       0.60      0.02      0.03       172
           1       0.72      1.00      0.84       723
           2       0.00      0.00      0.00        53
           3       0.00      0.00      0.00        54

    accuracy                           0.72      1002
   macro avg       0.33      0.25      0.22      1002
weighted avg       0.62      0.72      0.61      1002

==============sydneysiege==Veracity Report==============
             

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
# Overall veracity confusion matrix
from sklearn.metrics import confusion_matrix
y_true=[]
y_pred=[]
for lbl in ['charliehebdo','sydneysiege','germanwings-crash','ferguson','ottawashooting']:
  y_true.extend([y[0] for y in results['veracity'][lbl]])
  y_pred.extend([y[1] for y in results['veracity'][lbl]])
print('================Veracity Confucion matrix==============')
print(confusion_matrix(y_true,y_pred,normalize='true'))

================Veracity Confucion matrix==============
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


# Testing on unknown data

In [61]:
event_labels=['putinmissing','ebola-essien','prince-toronto']
def val_loader(val_index):
  for e in [l for i,l in enumerate(event_labels) if i==val_index]:
    with open('/content/drive/My Drive/DeepMoji/data_bert_{}.pkl'.format(e),'rb') as f:
      ev=pickle.load(f)
    yield ev,e

model=TreeLSTMCell(768,768)
model.load_state_dict(torch.load('/content/drive/My Drive/model_bert_balanced.pt'))
results={'stance':{'putinmissing':[],'ebola-essien':[],'prince-toronto':[]}, \
                    'veracity':{'putinmissing':[],'ebola-essien':[],'prince-toronto':[]}}
with torch.no_grad():
  for val_index in range(5):
    for event,lbl in val_loader(val_index):
      print('testing on {}...'.format(lbl))    
      for tree in event.treelist:
        nodes=tree[1]
        for i in range(len(nodes)):
          nodes[i].h=None
          nodes[i].c=None
        model.reset_h_c()  
        for i in range(len(nodes)):
          if(i!=(len(nodes)-1)):
            result=model(nodes[i])
            if(result is not None and nodes[i].y is not None):
              results['stance'][lbl].append((nodes[i].y,torch.argmax(result,dim=-1).item()))
          else:
            result=model(nodes[i],is_last=True) 
            results['veracity'][lbl].append((veracity_dict[str(tree[0])],torch.argmax(result,dim=-1).item()))

testing on putinmissing...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


KeyError: ignored

In [20]:
from sklearn.metrics import classification_report
for lbl in ['putinmissing','ebola-essien','prince-toronto']:
  y_true=[y[0] for y in results['veracity'][lbl]]
  y_pred=[y[1] for y in results['veracity'][lbl]]
  print('=============={}==Veracity Report=============='.format(lbl))
  print(classification_report(y_true,y_pred))

  y_true=[y[0] for y in results['stance'][lbl]]
  y_pred=[y[1] for y in results['stance'][lbl]]
  print('=============={}==Stance Report=============='.format(lbl))
  print(classification_report(y_true,y_pred)) 

==============putinmissing==Veracity Report==============
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         1
           2       1.00      0.25      0.40         8

    accuracy                           0.22         9
   macro avg       0.33      0.08      0.13         9
weighted avg       0.89      0.22      0.36         9

==============putinmissing==Stance Report==============
              precision    recall  f1-score   support

           0       0.22      0.18      0.20        11
           1       0.63      0.75      0.69        32
           2       0.33      0.20      0.25         5
           3       0.33      0.20      0.25         5

    accuracy                           0.53        53
   macro avg       0.38      0.33      0.35        53
weighted avg       0.49      0.53      0.50        53

==============ebola-essien==Veracity Report==============
            

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
# Overall veracity confusion matrix
from sklearn.metrics import confusion_matrix
y_true=[]
y_pred=[]
for lbl in ['putinmissing','ebola-essien','prince-toronto']:
  y_true.extend([y[0] for y in results['stance'][lbl]])
  y_pred.extend([y[1] for y in results['stance'][lbl]])
print('================stance Confucion matrix==============')
print(confusion_matrix(y_true,y_pred))

================stance Confucion matrix==============
[[ 7 14  1  5]
 [27 74  2 12]
 [ 2 11  2  2]
 [ 4 10  2  1]]


# Data distribution

In [27]:
ev_label_count=[]
for e in ['charliehebdo','sydneysiege','germanwings-crash','ferguson','ottawashooting']:
  with open('/content/drive/My Drive/DeepMoji/data_bert_{}.pkl'.format(e),'rb') as f:
    ev=pickle.load(f)
  label={0:0,1:0,2:0,3:0}   
  # for event in ev:
  for tree in ev.treelist:
    nodes=tree[1]
    for node in nodes:
      if(node.y is not None):
        label[node.y]+=1
  ev_label_count.append((e,label))    
print(ev_label_count)

[('charliehebdo', {0: 169, 1: 719, 2: 53, 3: 56}), ('sydneysiege', {0: 150, 1: 700, 2: 98, 3: 88}), ('germanwings-crash', {0: 46, 1: 172, 2: 28, 3: 10}), ('ferguson', {0: 138, 1: 713, 2: 99, 3: 90}), ('ottawashooting', {0: 105, 1: 477, 2: 63, 3: 74})]


In [44]:
ev_label_count=[]
for e in ['charliehebdo','sydneysiege','germanwings-crash','ferguson','ottawashooting']:
  with open('/content/drive/My Drive/DeepMoji/data_bert_{}_balanced.pkl'.format(e),'rb') as f:
    ev=pickle.load(f)
  label={0:0,1:0,2:0}   
  # for event in ev:
  for tree in ev.treelist:
    label[veracity_dict[str(tree[0])]]+=1
  ev_label_count.append((e,label))    
print(ev_label_count)

[('charliehebdo', {0: 39, 1: 39, 2: 39}), ('sydneysiege', {0: 51, 1: 51, 2: 51}), ('germanwings-crash', {0: 10, 1: 12, 2: 12}), ('ferguson', {0: 44, 1: 0, 2: 44}), ('ottawashooting', {0: 35, 1: 50, 2: 36})]
